In [1]:
from z3 import *
set_option(html_mode=False)

## Simplifying formulas

Find all parameters to control how formulas are simplified with `help_simplify()`.
$$
\begin{align}
x * 3 * x * x *x * 8 = 384\\
x = y+2 \\
(x+y)^2 \\
(x+y)(x-y)+2 = -x^2
\end{align}
$$

In [15]:
x,y,z = Reals("x y z")

print(simplify(x*3*x*x*x*8==384, mul_to_power=True))
print(simplify(x == y + 2, arith_lhs=True)) 
print(simplify((x + y)**2, som=True, mul_to_power=True))
print(simplify((x + y)*(x-y)+2 == -x**2, som=True, mul_to_power=True, arith_lhs=True))


x**4 == 16
x + -1*y == 2
x**2 + 2*x*y + y**2
2*x**2 + -1*y**2 == -2


In [16]:
A, B = Bools("A B")
print (simplify(And(A, B, True)))

print (simplify(And(A, False)))


And(A, B)
False


## Internal Representation
Expressions, sorts and declaration are internally abstract syntax trees.
Every expression has a sort.

Even though I called `Int("x")` a variable, it is internally a constant of type `IntSort`, represented as a function with $0$ arguments.

In [2]:
# The .sort() method retrieves the sort of an expression
x, y = Ints('x y')

print("x.sort():",x.sort())
print("(x + 1).sort():",(x+1).sort())
print("(x + 1.5).sort():",(x+1.5).sort())
print("(x > y).sort():",(x>y).sort())

x.sort(): Int
(x + 1).sort(): Int
(x + 1.5).sort(): Real
(x > y).sort(): Bool


### Symbolic-expression
[Wikipedia](https://en.wikipedia.org/wiki/S-expression)  
2 * (3+4)  
(* 2 (+ 3 4)) (prefix notation)  
S-expression as graph:
<div>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/6/6c/Corrected_S-expression_tree_2.svg/1024px-Corrected_S-expression_tree_2.svg.png" width="300"/>
</div>

In [4]:
# The method eq(n1,n2) returns True if n1 and n2 are structurally the same AST
x, y = Ints('x y')
print("(x+y) == (x+y)?")
print((x+y).sexpr(),"==",(x+y).sexpr())
print(eq(x + y, x + y))

print()
print("(x+y) == (y+x)?")
print((x+y).sexpr(),(y+x).sexpr())
print(eq(x + y, y + x))

(x+y) == (x+y)?
(+ x y) == (+ x y)
True
x + y == y + x

(x+y) == (y+x)?
(+ x y) (+ y x)
False


In [25]:
# If eq(n1,n2) returns True, then their hash values are the same
print((x + y).hash())
print((x + y).hash())
print((y + x).hash())

89782791
89782791
3776101329


## Arrays
Z3 provides a basic array type that maps from an index to a value.
`Select(a,i)` returns the value stored at position `i` of the array `a`.
`Store(a,i,v)` returns a new array. A copy of `a`, but with the value `v` at position `i`.

In python, `Select(a,i)` can be shortened to `a[i]`.

In [5]:
# A is an array mapping from integer to integer
A = Array("A", IntSort(), IntSort())
x = Int('x')

print (A[x])
print (Select(A, x))
print (Store(A, x, 10))

print()
print (Select(Store(A, 2, x+1), 2))
print (simplify(Select(Store(A, 2, x+1), 2)))

A[x]
A[x]
Store(A, x, 10)

Store(A, 2, x + 1)[2]
1 + x


Arrays in Z3 are generally used to model unbounded or very large arrays.  
It is usually much more efficient to create different variables using list comprehension for smaller arrays.

### Constant Arrays
`K(s,v)` costructs an array that maps all value of type `s` into an expression `v`.

In [8]:
AllZeroes = K(IntSort(),0)
OneHot = Store(AllZeroes,1_000_000,1)
print(OneHot)
x = Int("x")
s = Solver()
s.add(OneHot[x] == 1)

if s.check():
    print(s.model())

Store(K(Int, 0), 1000000000000, 1)
[x = 1000000000000]


## Advanced Datatypes


### List


In [10]:
def DeclareList(sort):
    List = Datatype('List_of_%s' % sort.name())
    List.declare('cons', ('head', sort), ('tail', List))
    List.declare('nil')
    return List.create()

IntList     = DeclareList(IntSort())
RealList    = DeclareList(RealSort())
IntListList = DeclareList(IntList)

l1 = IntList.cons(10, IntList.nil)
print (l1)
print (IntListList.cons(l1, IntListList.cons(l1, IntListList.nil)))
print (RealList.cons("1/3", RealList.nil))

print (l1.sort())

cons(10, nil)
cons(cons(10, nil), cons(cons(10, nil), nil))
cons(1/3, nil)
List_of_Int


### Enum
`EnumSort(name, [values])` creates new enumeration datatype with the given name. Each instance can have one of the values in `values`.
It returns the datatype and constants for each value.

In [12]:
Color, (red, green, blue) = EnumSort('Color', ('red', 'green', 'blue'))

In [13]:
print (green == blue)
print (simplify(green == blue))

c = Const('c', Color)
solve(c != green, c != blue)

green == blue
False
[c = red]
